In [19]:
%load_ext autoreload

%autoreload 2

In [20]:
from trace2segs import *


In [3]:
segs = gp.read_file('segments_dc.geojson')

In [10]:
import argparse

parser = argparse.ArgumentParser(description='find corresponding segments for each clip')

parser.add_argument('--segs', help='segments file', required=True)
parser.add_argument('-r', '--root-dir', type=str, help='root directory containing videos and gps traces, ' +
                                                   'it is the working directory(chdir to it)')
parser.add_argument('--match-file', default='gpx_video_match.csv',
                help='match between gpx, video and snapped file, it will be saved in the input-directory')
parser.add_argument('--segs-for-clips-csv', default='segs_for_clips.csv', help='segs_for clips')
parser.add_argument('--clips-quality-csv', default='clips_quality.csv', help='clips quality')
# test/debug argument, default False if not specified
parser.add_argument('-v', '--verbose', help='be verbose of the output', action='store_true')
parser.add_argument('--start-over', help='if specified, ignore existing trace2segs results', action='store_true')


args = parser.parse_args(['--segs', 'segments_dc.geojson'])

_StoreTrueAction(option_strings=['--start-over'], dest='start_over', nargs=0, const=True, default=False, type=None, choices=None, help='if specified, ignore existing trace2segs results', metavar=None)

In [4]:

match_file = pd.read_csv('gpx_video_match.csv')
snapped_files = match_file.json_file.values
segs_lin_res = []
clip_quality_res = []

In [37]:

def get_snapped_trace(segs, trace, close_jn_dist=10, far_jn_dist=30,bfr_crs=3559):

    snapped_trace = trace

    # find segment index for each point, keep #points without segment assignment
    snapped_trace_gpdf = gp.GeoDataFrame([Point(x) for x in snapped_trace], columns=['geometry'])
    pts_segs, pts_no_segs = pts2segs(snapped_trace_gpdf, segs, bfr_crs=bfr_crs,
                                     close_jn_dist=close_jn_dist, far_jn_dist=far_jn_dist)

    # get the segment index(indices) for each point(index) and merge into snapped_trace_gpdf
    pts_idx_ln_idx = snapped_trace_gpdf.merge(pts_segs, left_index=True, right_on='index_pt') \
        .groupby('index_pt')['index_ln'].apply(list).to_frame()
    snapped_trace_gpdf = snapped_trace_gpdf.merge(pts_idx_ln_idx, left_index=True, right_index=True)

    # 2 phrases seg_disambiguation
    snapped_trace_gpdf['clean_seg'] = seg_disambiguation(snapped_trace_gpdf.index_ln.values, keep_tie=True)
    snapped_trace_gpdf['clean_seg2'] = seg_disambiguation(snapped_trace_gpdf.clean_seg.values)
    return snapped_trace_gpdf

In [62]:

def one_video(args, segs, video_snapped):
    res = {}
    for snapped_res in video_snapped:
        clip_name = snapped_res['clip_name']

        if args.verbose:
            print('processing clip:', clip_name)

        # find segments for a clip, in terms of linear reference
        snapped_df = pd.DataFrame.from_dict(snapped_res['snapped'])
        no_snapped_pts = snapped_df.empty
        if not no_snapped_pts:
            snapped_trace = list(chain(*snapped_df.snapped.values))
            segs_res = get_snapped_trace(segs, snapped_trace)
            res[clip_name]=segs_res
    return res

In [63]:

processed_clips = {}
for sfile in snapped_files:
    if sfile!="split/DCIM/105_VIRB/VIRB0008.json": continue
    if not os.path.exists(sfile):
        if args.verbose:
            print('does not exist: %s' % sfile)
        continue

    print('processing clips of video:', sfile)
    video_snapped = load_json_file(sfile)
    res = one_video(args, segs, video_snapped)
    break

processing clips of video: split/DCIM/105_VIRB/VIRB0008.json


In [48]:
tmpdf = pd.read_csv('segs_for_clips.csv')

In [67]:
tmpdf[tmpdf.clip_name=='split/DCIM/105_VIRB/VIRB0008_025.MP4']

,Unnamed: 0,index_seg,start,end,ratio,length,ratio_before_round,clip_name
359,359,12567,0.3,0.8,0.5,34.331546,0.383750,split/DCIM/105_VIRB/VIRB0008_025.MP4
360,360,12568,0.0,0.6,0.6,101.268759,0.423558,split/DCIM/105_VIRB/VIRB0008_025.MP4


In [66]:
res['split/DCIM/105_VIRB/VIRB0008_025.MP4']

,geometry,index_ln,clean_seg,clean_seg2
0,POINT (-77.015365 38.887973),[12567],[12567],[12567]
1,POINT (-77.015365 38.887934),[12567],[12567],[12567]
2,POINT (-77.01536400000001 38.887894),[12567],[12567],[12567]
3,POINT (-77.01536400000001 38.887853),[12567],[12567],[12567]
4,POINT (-77.01536299999999 38.887816),[12567],[12567],[12567]
5,POINT (-77.01536299999999 38.88778),[12567],[12567],[12567]
6,POINT (-77.015362 38.887744),[12567],[12567],[12567]
7,POINT (-77.015362 38.887711),[12850],[12850],[12850]
8,POINT (-77.01536 38.887697),[12850],[12850],[12850]
9,POINT (-77.015356 38.887666),[12850],[12850],[12850]


In [52]:
tmpdf.groupby(['clip_name','index_seg']).count().sort_values('start')

Unnamed: 0  start  end  ratio  \
clip_name                            index_seg                                  
split/DCIM/103_VIRB/VIRB0007_000.MP4 808                 1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_003.MP4 9605                1      1    1      1   
                                     9606                1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_004.MP4 9603                1      1    1      1   
                                     9604                1      1    1      1   
                                     9605                1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_005.MP4 9603                1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_006.MP4 9603                1      1    1      1   
                                     12047               1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_007.MP4 12047               1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_000.MP4 3383                1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_002.MP4 9606                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_001.MP4 3383                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_002.MP4 3385                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_003.MP4 3385                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_004.MP4 3385                1      1    1      1   
                                     3386                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_005.MP4 3386                1      1    1      1   
                                     3387                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_006.MP4 3387                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_007.MP4 3387                1      1    1      1   
                                     3388                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_008.MP4 3388                1      1    1      1   
split/DCIM/105_VIRB/VIRB0022_001.MP4 3384                1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_002.MP4 6945                1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_001.MP4 6945                1      1    1      1   
split/DCIM/105_VIRB/VIRB0021_000.MP4 6945                1      1    1      1   
split/DCIM/105_VIRB/VIRB0020_009.MP4 9859                1      1    1      1   
                                     9860                1      1    1      1   
split/DCIM/105_VIRB/VIRB0020_010.MP4 9858                1      1    1      1   
...                                                    ...    ...  ...    ...   
split/DCIM/105_VIRB/VIRB0010_008.MP4 6215                1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_025.MP4 12567               1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_009.MP4 3382                1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_024.MP4 12942               1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_018.MP4 3149                1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_011.MP4 3380                1      1    1      1   
                                     3381                1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_012.MP4 3380                1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_013.MP4 98                  1      1    1      1   
                                     3380                1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_014.MP4 98                  1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_015.MP4 97                  1      1    1      1   
                                     98                  1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_016.MP4 97                  1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_017.MP4 3149                1      1    1      1   
split/DCIM/105_VIRB/VIRB0008_018.MP4 183                 1      1    1      1   
split/DCIM/105_VIRB/VIRB0041_003.MP4 745                 1      1    1      1  